<a href="https://colab.research.google.com/github/deutranium/tfx-addons/blob/main/tfx_addons/feature_selection/example/Palmer_Penguins_example_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFX Feature Selection Component

You may find the source code for the same [here](https://github.com/tensorflow/tfx-addons/tree/main/tfx_addons/feature_selection)

This example demonstrate the use of feature selection component. This project allows the user to select different algorithms for performing feature selection on datasets artifacts in TFX pipelines

Base code taken from: https://github.com/tensorflow/tfx/blob/master/docs/tutorials/tfx/components_keras.ipynb

## Setup
### Install TFX
_Note: In Google Colab, because of package updates, the first time you run this cell you must restart the runtime (Runtime > Restart runtime ...)._

In [ ]:
!pip install -U tfx

In [2]:
# getting the code directly from the repo
x = !pwd

if 'feature_selection' not in str(x):
  !git clone  -b main https://github.com/tensorflow/tfx-addons.git
  %cd tfx-addons/tfx_addons/feature_selection

Cloning into 'tfx-addons'...
remote: Enumerating objects: 2030, done.
remote: Counting objects: 100% (642/642), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 2030 (delta 408), reused 459 (delta 280), pack-reused 1388
Receiving objects: 100% (2030/2030), 66.24 MiB | 23.06 MiB/s, done.
Resolving deltas: 100% (1259/1259), done.
/content/tfx-addons/tfx_addons/feature_selection


### Import packages
Importing the necessary packages, including the standard TFX component classes

In [3]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
import importlib
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
import importlib
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

# importing the feature selection component
from component import FeatureSelection


# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

## Palmer Penguins example pipeline

### Download Example Data

We download the example dataset for use in our TFX pipeline.

The dataset we're using is the [Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) which is also used in other
[TFX examples](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

There are four numeric features in this dataset:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

All features were already normalized to have range [0,1]. We will build a
that selects 2 features to be eliminated from the dataset in other to improve the performance of the mode in predicting the `species` of penguins.

In [5]:
# getting the dataset
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
             
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datajreimlm6/data.csv', <http.client.HTTPMessage at 0x7fbbd237c110>)

### Run TFX Components
In the cells that follow, we create TFX components one-by-one and generates `example` using `exampleGen` component.

In [7]:
context = InteractiveContext()

#create and run exampleGen component
example_gen = CsvExampleGen(input_base=_data_root )
context.run(example_gen)

#create and run statisticsGen component
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
# using the feature selection component
#feature selection component

feature_selector = FeatureSelection(orig_examples = example_gen.outputs['examples'],
                                   module_file='example.modules.penguins_module')
context.run(feature_selector)

ExecutionResult(
    component_id: FeatureSelection
    execution_id: 3
    outputs:
        feature_selection: OutputChannel(artifact_type=Feature Selection, producer_component_id=FeatureSelection, output_key=feature_selection, additional_properties={}, additional_custom_properties={})
        updated_data: OutputChannel(artifact_type=Examples, producer_component_id=FeatureSelection, output_key=updated_data, additional_properties={}, additional_custom_properties={}))

In [12]:
# Display Selected Features
context.show(feature_selector.outputs['feature_selection']._artifacts[0])

Artifact(artifact: id: 3
type_id: 18
uri: "/tmp/tfx-interactive-2022-03-30T15_07_37.697636-sx36olma/FeatureSelection/feature_selection/3"
properties {
  key: "p_values"
  value {
    struct_value {
      fields {
        key: "body_mass_g"
        value {
          number_value: 4.195541966407283e-05
        }
      }
      fields {
        key: "culmen_depth_mm"
        value {
          number_value: 8.747194971213637e-05
        }
      }
      fields {
        key: "culmen_length_mm"
        value {
          number_value: 0.0007791004116491523
        }
      }
      fields {
        key: "flipper_length_mm"
        value {
          number_value: 3.104844357027674e-05
        }
      }
    }
  }
}
properties {
  key: "scores"
  value {
    struct_value {
      fields {
        key: "body_mass_g"
        value {
          number_value: 20.15780587994667
        }
      }
      fields {
        key: "culmen_depth_mm"
        value {
          number_value: 18.68838478142841
        }
      }
      fields {
        key: "culmen_length_mm"
        value {
          number_value: 14.314741244525251
        }
      }
      fields {
        key: "flipper_length_mm"
        value {
          number_value: 20.7599237548046
        }
      }
    }
  }
}
properties {
  key: "selected_features"
  value {
    struct_value {
      fields {
        key: "__value__"
        value {
          list_value {
            values {
              string_value: "flipper_length_mm"
            }
            values {
              string_value: "body_mass_g"
            }
          }
        }
      }
    }
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "feature_selection"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "FeatureSelection"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.7.0"
  }
}
state: LIVE
, artifact_type: id: 18
name: "Feature Selection"
properties {
  key: "p_values"
  value: STRUCT
}
properties {
  key: "scores"
  value: STRUCT
}
properties {
  key: "selected_features"
  value: STRUCT
}
)

As seen above, `.selected_features` contains the features selected after running the component with the speified parameters.

To get the info about updated `Example` artifact, one can view it as follows:

In [16]:
context.show(feature_selector.outputs['updated_data']._artifacts[0])

.type,<class 'tfx.types.standard_artifacts.Examples'>
.uri,/tmp/tfx-interactive-2022-03-30T15_07_37.697636-sx36olma/FeatureSelection/updated_data/3
.span,0
.split_names,"[""train"", ""eval""]"
.version,0
